In [167]:
import sys
import pandas as pd
import ipysheet
sys.path.append('../src')
from restart import NoteCompose
from util import set_config, to_df, to_sheet, format_population, display_population

config = set_config('../src')
restart = NoteCompose(configdir='../src/', population='oes', demand='washington', subpop='wa_tier2_opt1', state='Washington')
population = restart.model.population
resource = restart.model.resource
demand = restart.model.demand
pop_sheet = format_population(to_sheet(population.detail_pd_df))
burn_sheet = to_sheet(demand.level_to_res_mn_df)

Here we define a function called `set_stock()` that will get called upon user interaction. 

It does the math to update the stockpile, and then displays the data in a custom-formatted `ipysheet`.

In [2]:
import ipywidgets as widgets
import ipyvuetify as v

def set_stock(days):
    resource.demand(resource.inventory_ln_df)
    resource.set_inv_min(demand.level_total_demand_ln_df, days)
    stockpile = to_sheet(resource.inventory_ln_df)
    stockpile_cost_df = resource.inventory_ln_df.multiply(resource.cost_ln_df).round()
    stockpile_cost = to_sheet(stockpile_cost_df)
    display_population(stockpile)
    display_population(stockpile_cost, money=True)

This is the actual interactivity. The back-end of it is an `ipywidgets` slider that we've tied to the `set_stock()` function using `widgets.interactive_output`. 

However, since the Vuetify slider is prettier, that is actually what the user sees - the `ipywidgets` slider is invisible, but the Vuetify slider updates it behind the scenes.

In [1]:
v_slider = v.Slider(
    _metadata={'mount_id': 'histogram_bins2'}, 
    thumb_label='always', 
    thumb_size=17, 
    class_='px-4', 
    v_model=30)
slider = widgets.IntSlider()
widgets.link((v_slider, 'v_model'), (slider, 'value'))

out = widgets.interactive_output(set_stock, {'days': slider})

NameError: name 'v' is not defined

This is how you can specify the layout of the webpage. All of this actually gets transpiled to HTML at runtime, so think of all the Python here as wrappers for HTML. Every UI component in Vuetify can be translated to Python in a predictable way; see the Vuetify docs and ipyvuetify docs that are linked below.

Vuetify: https://vuetifyjs.com/en/components/api-explorer/

ipyvuetify: https://ipyvuetify.readthedocs.io/en/latest/

In [197]:
import ipyvuetify as v
v.theme.themes.light.primary = 'colors.teal'

v.Tabs(_metadata={'mount_id': 'content-main'}, children=[
    v.Tab(children=['Stockpile Projections']),
    v.Tab(children=['Population Details']),
    v.Tab(children=['Burn Rates']),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, xl4=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Days of Stockpile"]),
                v.CardSubtitle(children=["Drag the slider to adjust stockpile days"]),
                v_slider, out
            ])
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_center=True, children=[
            v.Card(xs12=True, lg6=True, xl4=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Population Details"]),
                v.CardSubtitle(children=["Attributes of the selected populations"]),
                pop_sheet
            ])
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_center=True, children=[
            v.Card(xs12=True, lg6=True, x14=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Burn Rates"]),
                v.CardSubtitle(children=["Resource consumption per day"]),
                burn_sheet
            ])
        ])
    ])
])

Tabs(children=[Tab(children=['Stockpile Projections']), Tab(children=['Population Details']), Tab(children=['B…